### Word Frequency

In [124]:
import string
import pandas as pd
import nltk 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
translator = str.maketrans('', '', string.punctuation)
from functools import reduce

[nltk_data] Downloading package stopwords to /Users/matt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/matt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/matt/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [126]:
postsDF = pd.read_csv('./scrappy/scrappy_do.csv')

def wordCount(post):
    d = {}
    l = str(post).split(" ")
    for w in l:
        w = w.translate(translator) # remove punctuation
        w = w.lower()
        if (d.get(w)):
            d[w] = d[w] + 1
        else:
            d[w] = 1
    return d

def stopUnique(post):
    wc = wordCount(post)
    unique = list(wc.keys())
    filtered = [w for w in unique if w not in stop_words]
    return filtered

nouns = ['NN', 'NNS', 'NNP', 'NNPS']

postsDF['words'] = postsDF['message'].map(lambda x: stopUnique(x))
# # postsDF.head()

combined = postsDF['words'].map(lambda x: ' '.join(x))
oneString = ' '.join(combined)
# allCount = wordCount(oneString)
# allCount

tokens = nltk.word_tokenize(oneString)
tagged = nltk.pos_tag(tokens)


In [162]:
n = filter(lambda x: x[1] in nouns, tagged)
arr = list(n)
finalNouns = ' '.join(list(map(lambda x: x[0], arr)))
f = wordCount(finalNouns)

test = pd.DataFrame.from_dict(f, orient='index')
importantWords = list(test.sort_values(by=0, ascending=False).head(2000).index)

In [163]:
def onlyImportant(postList):
    f = filter(lambda w: w in importantWords, postList)
    return list(f)

removed = list(map(onlyImportant, list(postsDF['words'])))
postsDF['filtered'] = removed
postsDF.head()

,date,userid,message,words,filtered
0,2002-03-25T05:54:02+00:00,merc1,I personally think that with a few tweaks the ...,"[personally, think, tweaks, c320, could, also,...","[think, sit, top, group, entrylevel, luxury, s..."
1,2002-03-25T07:06:29+00:00,fredvh,I am debating a new purchase and these two are...,"[debating, new, purchase, two, top, list, pric...","[purchase, top, list, price, difference, acura..."
2,2002-03-25T17:02:27+00:00,blueguydotcom,"Great handling, RWD, excellent engine and the ...","[great, handling, rwd, excellent, engine, best...","[handling, rwd, excellent, engine, warranty, w..."
3,2002-03-25T23:04:37+00:00,hungrywhale,And no manual tranny. That may not matter to y...,"[manual, tranny, may, matter, certainly, many,...","[manual, tranny, matter, enthusiast, imo, boar..."
4,2002-03-26T00:44:13+00:00,riez,One beauty of BMW 3 Series is that there are s...,"[one, beauty, bmw, 3, series, many, models, ch...","[beauty, bmw, series, models, choose, half, bu..."


In [3]:
dataset = list(postsDF['words'])

In [22]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.09, use_colnames=True)


frequent_itemsets


,support,itemsets
0,0.297590,()
1,0.112260,(3)
2,0.111060,(also)
3,0.126708,(better)
4,0.211272,(bmw)
5,0.355982,(car)
6,0.220504,(cars)
7,0.097904,(could)
8,0.188654,(dont)
9,0.123615,(drive)


In [24]:
from mlxtend.frequent_patterns import association_rules

result_lift = association_rules(frequent_itemsets1, metric="confidence", min_threshold=0.1)


In [26]:
result_lift.sort_values(by=['lift'],ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
13,(cars),(car),0.220504,0.355982,0.123523,0.560184,1.573630,0.045027,1.464290
12,(car),(cars),0.355982,0.220504,0.123523,0.346992,1.573630,0.045027,1.193700
24,(would),(car),0.216119,0.355982,0.118861,0.549979,1.544961,0.041926,1.431083
25,(car),(would),0.355982,0.216119,0.118861,0.333895,1.544961,0.041926,1.176814
20,(car),(one),0.355982,0.227705,0.123708,0.347510,1.526143,0.042649,1.183613
21,(one),(car),0.227705,0.355982,0.123708,0.543280,1.526143,0.042649,1.410093
16,(get),(car),0.199086,0.355982,0.108013,0.542546,1.524081,0.037142,1.407830
17,(car),(get),0.355982,0.199086,0.108013,0.303423,1.524081,0.037142,1.149786
15,(car),(dont),0.355982,0.188654,0.102336,0.287474,1.523817,0.035178,1.138690
14,(dont),(car),0.188654,0.355982,0.102336,0.542452,1.523817,0.035178,1.407540
